In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def natural_sort_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', s)]


In [ ]:
import os
import cv2 
import re

image_folder = "/kaggle/input/millers-planet/train/train"

images =[] # empty list to store the training data
image_names = [] # empty list for storing filenames

filenames = sorted(os.listdir(image_folder), key = natural_sort_key)
for filename in filenames:
    if filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename) # creating a dynamic path to the image
        # cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) images already in grayscale ->256 x 256
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        images.append(image)
        image_names.append(filename)
        

In [ ]:
print(image_names)

In [ ]:
print(images[1].shape) # proof that everything is now in grayscale

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1, 5, i+1)  # Create a subplot (1 row, 5 columns)
    plt.imshow(images[i], cmap='gray')  # Show the image in grayscale
    plt.axis('off')  # Hide axes
    plt.title(image_names[i])

plt.show()

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1,5,i+1)
    
    blur = cv2.GaussianBlur(images[i], (7,7), cv2.BORDER_DEFAULT)
    
    canny = cv2.Canny(blur, 100, 150)
    plt.imshow(canny)
    plt.axis('off')
    plt.title(image_names[i])

plt.show()

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1,5,i+1)
    
    canny = cv2.Canny(images[i], 100, 150)
    plt.imshow(canny)
    plt.axis('off')
    plt.title(image_names[i])

plt.show()

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1,5,i+1)
    
    blur = cv2.GaussianBlur(images[i], (5,5), cv2.BORDER_DEFAULT)
    canny = cv2.Canny(blur, 100, 150)

    contours, hierarchies = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(canny)
    plt.axis('off')
    plt.title(image_names[i])
    print(count_squaress(contours))

plt.show()

In [ ]:
def count_squaress(contours):
    squares = 0
    for contour in contours:
        # Skip very small contours
        if cv2.contourArea(contour) < 10:  # Adjust based on your image scale
            continue
            
        # Approximate the contour with relaxed epsilon
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.045 * peri, True)  # Increased from 0.02 to 0.04
        
        # Relaxed square detection:
        # - Looking for 4-sided polygons (rectangles/squares)
        # - Relaxed aspect ratio constraints
        # - Must still be convex
        if len(approx) == 4 and cv2.isContourConvex(approx):
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            
            # More relaxed aspect ratio (0.7 to 1.3)
            if 0.85 <= aspect_ratio <= 1.15:
                squares += 1

    return squares

In [ ]:
def is_square(cnt):
    if len(cnt) != 4:
        return False
    angles = []
    for i in range(4):
        pt1 = cnt[i][0]
        pt2 = cnt[(i+1) % 4][0]
        pt3 = cnt[(i+2) % 4][0]
        angle = angle_between(pt1, pt2, pt3)
        angles.append(angle)
    return all(80 <= angle <= 100 for angle in angles)

In [ ]:
def count_squares3(contours):
    squares = 0
    for contour in contours:
        # Skip very small contours
        if cv2.contourArea(contour) < 25:  # Adjust based on your image scale
            continue
            
        # Approximate the contour with relaxed epsilon
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.045 * peri, True)  # Increased from 0.02 to 0.04
        
        # Relaxed square detection:
        # - Looking for 4-sided polygons (rectangles/squares)
        # - Relaxed aspect ratio constraints
        # - Must still be convex
        if len(approx) == 4 and cv2.isContourConvex(approx):
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            
            # More relaxed aspect ratio (0.7 to 1.3)
            if 0.85 <= aspect_ratio <= 1.15:
                if is_square(approx):
                    squares += 1

    return squares

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1,5,i+1)
    
    blur = cv2.GaussianBlur(images[i], (5,5), cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(images[i],125,255,cv2.THRESH_BINARY)
    
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(thresh)
    plt.axis('off')
    plt.title(image_names[i])
    print(count_squaress(contours))

plt.show()

In [ ]:
plt.figure(figsize = (10,5))

for i in range(5):
    plt.subplot(1,5,i+1)
    
    blur = cv2.GaussianBlur(images[i], (5,5), cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(blur,125,255,cv2.THRESH_BINARY)
    
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(thresh)
    plt.axis('off')
    plt.title(image_names[i])
    print(count_squares3(contours))

plt.show()

In [ ]:
y_preds = []
for image in images:
    blur = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(blur,125,255,cv2.THRESH_BINARY)

    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    y_preds.append(count_squaress(contours))



In [ ]:
from sklearn.metrics import accuracy_score
df = pd.read_csv("/kaggle/input/millers-planet/train_gt.csv")
y_true = df["square_count"]


accuracy_score(y_true, y_preds)

In [ ]:
y_preds = []
for image in images:
    blur = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(blur,125,255,cv2.THRESH_BINARY)
    
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    y_preds.append(count_squares3(contours))



In [ ]:
from sklearn.metrics import accuracy_score
df = pd.read_csv("/kaggle/input/millers-planet/train_gt.csv")
y_true = df["square_count"]


accuracy_score(y_true, y_preds)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(y_true, y_preds)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


NOW WE TRY TO TEST 

In [ ]:
test_folder = "/kaggle/input/millers-planet/test/test"

test_images= []
image_names = []

filenames = sorted(os.listdir(test_folder) , key = natural_sort_key)
for filename in filenames:
    if(filename.endswith(".png")):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        test_images.append(image)
        image_names.append(filename)


In [ ]:
plt.figure(figsize = (10,10))

for i in range(10):
    plt.subplot(1,10,i+1)
    
    blur = cv2.GaussianBlur(test_images[i], (5,5), cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(blur,125,255,cv2.THRESH_BINARY)
    
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(thresh)
    plt.axis('off')
    plt.title(image_names[i])
    print(count_squares3(contours))

plt.show()